In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

gh_topic_url = 'https://github.com/topics'

url_response = requests.get(gh_topic_url)

topic_text = url_response.text

with open('downloaded_file.html', 'w', encoding = 'utf-8') as f:
    f.write(topic_text)

topic_doc = BeautifulSoup(topic_text, 'html.parser')

url_class = "f3 lh-condensed mb-0 mt-1 Link--primary"
topic_name = topic_doc.find_all('p', class_ = url_class)

TopicList = [] 
for topic in topic_name:
    text = str(topic.get_text())
    append = TopicList.append(text)

topic_class = "f5 color-fg-muted mb-0 mt-1"
topic_desc = topic_doc.find_all('p', class_ = topic_class)

TopicDesc = []
for desc in topic_desc:
    text = str(desc.get_text())
    append = text.strip()
    f_append = TopicDesc.append(append)

sl_class = 'no-underline flex-grow-0'
sub_link = topic_doc.find_all('a', class_ = sl_class)

TopicUrl = []

base_url = 'https://github.com'

for link in sub_link:
    TopicUrl.append(base_url + link['href'])

topic_dict = {
    'Title' : TopicList,
    'Description' : TopicDesc,
    'URL' : TopicUrl    
}

Topics_df = pd.DataFrame(topic_dict)

Topics_df.to_csv('GHTopics', index = None)

topic_url0 = TopicUrl[0]
response = requests.get(topic_url0)
Topic_doc = BeautifulSoup(response.text, 'html.parser')
Txt = Topic_doc #SoupObject

user_tags = 'f3 color-fg-muted text-normal lh-condensed'
repo_tags = Txt.find_all('h3', class_ = user_tags)

h3_tags = repo_tags[0].find_all('a')

auth_name = h3_tags[0].text.strip()

repository_name = h3_tags[1].text.strip()

repository_url = base_url + h3_tags[1]['href']

s_class = 'Counter js-social-count'
repo_stars = Topic_doc.find_all('span', class_ = s_class)

stars = repo_stars[0]['title']

def repo_info(repo_tags, star_tag):
    h_tags = repo_tags.find_all('a')
    author = h_tags[0].text.strip()
    repo_name = h_tags[1].text.strip()
    repo_url = base_url + h_tags[1]['href']
    star_count = star_tag['title']
    return author,repo_name,repo_url,star_count

repo_info(repo_tags[0],repo_stars[0])

topic_repo_dict = {
    "Author" : [],
    "Repo_name": [],
    'Repo_Url' : [],
    "Stars" : []
}

for i in range(len(repo_tags)):
    repo_infor = repo_info(repo_tags[i], repo_stars[i])
    topic_repo_dict['Author'].append(repo_infor[0])
    topic_repo_dict['Repo_name'].append(repo_infor[1])
    topic_repo_dict['Repo_Url'].append(repo_infor[2])
    topic_repo_dict['Stars'].append(repo_infor[3])
    
topic_repoDF = pd.DataFrame(topic_repo_dict)

# Functions

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://github.com'

def get_topic_page(topic_url):
    # Download the page
    url_response = requests.get(topic_url)
    # Check the response
    if url_response.status_code != 200:
        raise Exception(f"Failed to load page {topic_url}")
    # Parse using BeautifulSoup
    topic_doc = BeautifulSoup(url_response.text, 'html.parser')
    return topic_doc

def get_repo_info(repo_tags, star_tag):
    h_tags = repo_tags.find_all('a')
    author = h_tags[0].text.strip()
    repo_name = h_tags[1].text.strip()
    repo_url = base_url + h_tags[1]['href']
    star_count = star_tag['title']
    return author, repo_name, repo_url, star_count

def get_topic_repos(topic_doc):
    # Get h3 tags containing repo title, URL, and username
    user_tags = 'f3 color-fg-muted text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h3', class_=user_tags)
    # Get Star Tags
    s_class = 'Counter js-social-count'
    repo_stars = topic_doc.find_all('span', class_=s_class)
    
    if not repo_stars:
        # Handle the case where no stars are found
        print("No star information found.")
        return None
    
    stars = repo_stars[0]['title']
    
    topic_repo_dict = {"Author": [], "Repo_name": [], 'Repo_Url': [], "Stars": []}
    
    # Get repo info
    for i in range(len(repo_tags)):
        repo_info_result = get_repo_info(repo_tags[i], repo_stars[i])
        topic_repo_dict['Author'].append(repo_info_result[0])
        topic_repo_dict['Repo_name'].append(repo_info_result[1])
        topic_repo_dict['Repo_Url'].append(repo_info_result[2])
        topic_repo_dict['Stars'].append(repo_info_result[3])
    
    return pd.DataFrame(topic_repo_dict)

def get_topic_names(topic_doc):
    url_class = "f3 lh-condensed mb-0 mt-1 Link--primary"
    topic_name = topic_doc.find_all('p', class_=url_class)
    TopicList = [] 
    for topic in topic_name:
        text = str(topic.get_text())
        append = TopicList.append(text)
    return TopicList

def get_topic_description(topic_doc):
    topic_class = "f5 color-fg-muted mb-0 mt-1"
    topic_desc = topic_doc.find_all('p', class_=topic_class)

    TopicDesc = []
    for desc in topic_desc:
        text = str(desc.get_text())
        append = text.strip()
        f_append = TopicDesc.append(append)
    return TopicDesc

def get_topic_urls(topic_doc):
    sl_class = 'no-underline flex-grow-0'
    sub_link = topic_doc.find_all('a', class_=sl_class)

    TopicUrl = []

    for link in sub_link:
        TopicUrl.append(base_url + link['href'])
    return TopicUrl

def scrape_topics(topic_url, destination):
    response = requests.get(topic_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    topics_dict = {
        'title': get_topic_names(topic_doc),
        'description': get_topic_description(topic_doc),
        'url': get_topic_urls(topic_doc)
    }
    return pd.DataFrame(topics_dict)

def scrape_topics_repos():
    print('Scraping list of topics')
    topics_df = scrape_topics(base_url + '/topics', 'data/topics.csv')
    
    os.makedirs('data', exist_ok=True)
    
    if topics_df is None:
        return
    
    for index, row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_topics(row['url'], 'data/{}.csv'.format(row['title']))

#scrape_topics_repos()